In [1]:
import pandas as pd

In [3]:
data=pd.read_csv("updatedTweets.csv")
data.head()

,id,text,created_at,author_id,in_reply_to_user_id,lang,author,entities,referenced_tweets,media,geo
0,1538914496529448960,"RT @AzimioUmoja: Tumaini, Rongai town, Kajiado...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1538914403789471746,@hsriiix @fiziedan Mende gila? Hahaha.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1538914385531482113,RT @Kenyans: Stickers with famous Kenyan phras...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1538914260260102145,RT @soyrafillo: Y la chula esta que ya esta ac...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1538914144224792582,RT @rootsparty01: Mr 6 and Mr kitendawili Waja...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df=data[['id','text']]
df.head()

,id,text
0,1538914496529448960,"RT @AzimioUmoja: Tumaini, Rongai town, Kajiado..."
1,1538914403789471746,@hsriiix @fiziedan Mende gila? Hahaha.
2,1538914385531482113,RT @Kenyans: Stickers with famous Kenyan phras...
3,1538914260260102145,RT @soyrafillo: Y la chula esta que ya esta ac...
4,1538914144224792582,RT @rootsparty01: Mr 6 and Mr kitendawili Waja...


In [5]:
#preprocess text
df.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
324200     True
324201     True
324202     True
324203     True
324204     True
Length: 324205, dtype: bool

In [6]:
df1=df.drop_duplicates()

In [7]:
df1

,id,text
0,1538914496529448960,"RT @AzimioUmoja: Tumaini, Rongai town, Kajiado..."
1,1538914403789471746,@hsriiix @fiziedan Mende gila? Hahaha.
2,1538914385531482113,RT @Kenyans: Stickers with famous Kenyan phras...
3,1538914260260102145,RT @soyrafillo: Y la chula esta que ya esta ac...
4,1538914144224792582,RT @rootsparty01: Mr 6 and Mr kitendawili Waja...
...,...,...
323700,1509683872300208139,@GasparyCharlesG @yose_hoza @ahmedkherlid @dr_...
323701,1509683844013723650,Escribí estas notas sobre los orígenes del coo...
323702,1509683489616154625,@estoriboris Les damos a Novoa y un abracito p...
323703,1509683190042173447,@ElLorko Quien se perdería de dirigir a CHUNGA


In [8]:
!pip install gensim

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.21
    Uninstalling Cython-0.29.21:
      Successfully uninstalled Cython-0.29.21


In [18]:
import re
import gensim
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Immeculate\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [19]:
punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'         # define a string of punctuation symbols

In [20]:
# Functions to clean tweets
def remove_links(tweet):
    """Takes a string and removes web links from it"""
    tweet = re.sub(r'http\S+', '', tweet)   # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # remove bitly links
    tweet = tweet.strip('[link]')   # remove [links]
    tweet = re.sub(r'pic.twitter\S+','', tweet)
    return tweet

In [21]:
def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove tweeted at
    return tweet

In [22]:

def remove_hashtags(tweet):
    """Takes a string and removes any hash tags"""
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove hash tags
    return tweet

def remove_av(tweet):
    """Takes a string and removes AUDIO/VIDEO tags or labels"""
    tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    return tweet

def tokenize(tweet):
    """Returns tokenized representation of words in lemma form excluding stopwords"""
    result = []
    for token in gensim.utils.simple_preprocess(tweet):
        if token not in gensim.parsing.preprocessing.STOPWORDS \
                and len(token) > 2:  # drops words with less than 3 characters
            result.append(lemmatize(token))
    return result


In [23]:
def lemmatize(token):
    """Returns lemmatization of a token"""
    return WordNetLemmatizer().lemmatize(token, pos='v')

def preprocess_tweet(tweet):
    """Main master function to clean tweets, stripping noisy characters, and tokenizing use lemmatization"""
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet_token_list = tokenize(tweet)  # apply lemmatization and tokenization
    tweet = ' '.join(tweet_token_list)
    return tweet

def basic_clean(tweet):
    """Main master function to clean tweets only without tokenization or removal of stopwords"""
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet = re.sub('📝 …', '', tweet)
    return tweet

In [24]:
def tokenize_tweets(df):
    """Main function to read in and return cleaned and preprocessed dataframe.
    This can be used in Jupyter notebooks by importing this module and calling the tokenize_tweets() function
    Args:
        df = data frame object to apply cleaning to
    Returns:
        pandas data frame with cleaned tokens
    """

    df['tokens'] = df['text'].apply(preprocess_tweet)
    num_tweets = len(df)
    print('Complete. Number of Tweets that have been cleaned and tokenized : {}'.format(num_tweets))
    return df

In [25]:
tokenize_tweets(df1)

Complete. Number of Tweets that have been cleaned and tokenized : 323705


<ipython-input-24-6c0165a07ee9>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'] = df['text'].apply(preprocess_tweet)


,id,text,tokens
0,1538914496529448960,"RT @AzimioUmoja: Tumaini, Rongai town, Kajiado...",tumaini rongai town kajiado county
1,1538914403789471746,@hsriiix @fiziedan Mende gila? Hahaha.,mende gila hahaha
2,1538914385531482113,RT @Kenyans: Stickers with famous Kenyan phras...,stickers famous kenyan phrase mimi sipangwingw...
3,1538914260260102145,RT @soyrafillo: Y la chula esta que ya esta ac...,chula esta que esta acabando grabar capitulo s...
4,1538914144224792582,RT @rootsparty01: Mr 6 and Mr kitendawili Waja...,kitendawili wajackoya unstoppable
...,...,...,...
323700,1509683872300208139,@GasparyCharlesG @yose_hoza @ahmedkherlid @dr_...,jack gibson waandishi kama kitenge huganya kaz...
323701,1509683844013723650,Escribí estas notas sobre los orígenes del coo...,escribí estas notas sobre los orígenes del coo...
323702,1509683489616154625,@estoriboris Les damos a Novoa y un abracito p...,les damos novoa abracito por chunga
323703,1509683190042173447,@ElLorko Quien se perdería de dirigir a CHUNGA,quien perdería dirigir chunga


In [ ]:
df1

In [ ]:
import re
list_of_hate_words=['mende','sipangwingwi','madoadoa','wabara','wakuja']
df1["Label"]=df1["tokens"].map(lambda x: "Hate" if (("mende" or "sipangwingwi" or "madoadoa" or "wabara" or "wakuja") in x) else 'No Hate')
#df1["Label"]=df1.apply(lambda x: 'Hate' if x==[df1['tokens'].str.contains('mende')|df1['tokens'].str.contains('sipangwingwi')|df1['tokens'].str.contains('wabara')|df1['tokens'].str.contains('madoadoa')|df1['tokens'].str.contains('wakuja') else "No Hate"]

In [ ]:
df1

In [ ]:
df1['Label'].value_counts()

In [ ]:
df1['Label'].value_counts()

In [ ]:
df1['label_category']=df1['Label'].apply(lambda x:'1' if x=='Hate' else '0') #'hate'=1 'not hate'=0

In [ ]:
df1

In [ ]:
df1['label_category'].value_counts()